# Trade
This notebook trades with the hyperparameters selected in the previous over the full set of trading data.

Note that in reality the byperparameter tunning could be repeated every trading day but we will ignore that complication.

#### Import necessary modules

In [7]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import boto3
from IPython.display import Image
import math
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
from sagemaker.session import Session
import seaborn as sns; sns.set()
import shutil
from time import sleep

#### Set for local parameters

In [8]:
n_instances = 1
image_type = 'cpu'
instance_type = 'ml.m5.large'

hyperparameters = {'start_day': 504,
                   'days_per_epoch': 40,
                   'fc1': 9,
                   'fc2': 0,
                   'lr_actor': 0.00013,
                   'lr_critic': 0.0040}

train_use_spot_instances = True
train_max_run = 14400
train_max_wait = 14400 if train_use_spot_instances else None

sagemaker_session = Session()
bucket_name = sagemaker_session.default_bucket()
role = get_execution_role()
account = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name
image_name = '{}.dkr.ecr.{}.amazonaws.com/portfolio-optimization-{}:latest'.format(account, region, image_type)

#### Submit the job

In [9]:
estimator = Estimator(role=role,
                      train_instance_count=n_instances,
                      train_instance_type=instance_type,
                      image_name=image_name,
                      hyperparameters=hyperparameters)
estimator.fit()


2020-07-12 19:06:02 Starting - Starting the training job...
2020-07-12 19:06:04 Starting - Launching requested ML instances.........
2020-07-12 19:07:43 Starting - Preparing the instances for training...
2020-07-12 19:08:29 Downloading - Downloading input data...
2020-07-12 19:08:35 Training - Downloading the training image...
2020-07-12 19:09:26 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-07-12 19:09:26,460 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-07-12 19:09:26,463 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-07-12 19:09:26,474 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-07-12 19:09:29,520 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-07-12 19:09:29,520 sagemaker-containers IN

KeyboardInterrupt: 

#### Get the results

In [ ]:
s3 = boto3.resource('s3')
key = '{}/output/output.tar.gz'.format(estimator._current_job_name)
s3.Bucket(bucket).download_file(key, 'output.tar.gz')
shutil.unpack_archive('output.tar.gz')
Image(filename='history.png') 

# Reference
- [Tennis Sensitivity](https://github.com/daniel-fudge/sagemaker-tennis/blob/master/sensitivity.ipynb)
- [Tennis Repo](https://github.com/daniel-fudge/sagemaker-tennis)
#### SageMaker
- [SageMaker Instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/)
- [SageMaker Instance prices](https://aws.amazon.com/sagemaker/pricing/)
- [SageMaker Estimator SDK](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)